# <center>DATA_SCI 423 | Final Project</center>

# <center>Understanding Alloy Steel Composition-Property <br/> Relationships Using Machine Learning<center>

## <center> Raymond Wang, Yangdongling Liu </center>

## I. Introduction

We investigate the composition-property relationships in alloy steels using machine learning models. Alloys that consist of multi-elemental composition exhibit a large variety of physical properties, and have been widely used in modern industry. For instance, copper alloys are commonly used in the manufacture of electrical equipment, titanium alloys are most often seem in biomaterial applications. However, the high dimensional composition space makes it challenging to understand how elemental composition govern alloy properties. Existing theoretical models such as finite element model (FEM) are in general computationally expensive. In this work, we present a reliable and efficient way to directly predict alloy steel properties from their elemental composition using machine learning.

The workflow of this project is shown in the figure below. We obtain 855 alloy steel data from unstructured online resources. After data preprocessing, we first analyze correlation within the dataset, where several strongly correlated quantities are identified (e.g. hardness and tensile strength). Then we benchmark the performance of 10 machine learning algorithms. Extreme gradient boosting (XGBoost) outperforms the other models in our case. Further analysis shows that predicting thermal conductivity from chemical composition using XGBoost has satisfying accuracy and the model performance could be improved by having more training data. 

Our findings suggest that machine learning is a powerful tool that could provide more insights of alloy steel composition-property relationship than using human physical intuition or experience, and that data-driven research is a thriving division in physical science community.

<img src="workflow.png" alt="drawing" width="500"/>

## II. Data Acquisition and Preprocessing

Alloy steel data is collected froma [Matweb](http://www.matweb.com/index.aspx), where alloy steels containing Manganese, Chromium, and Nickel are set as the target materials for scraping. The scraping code automatically collects materials information that meets the searching criteria and saves to local CSV files. Physical properties (e.g. hardness, thermal conductivity) in different units, chemical compositions (e.g. per centage weight of Mn, Ni, Cr), and the potentially helpful comments, are saved to local files with unchanged format. Since there is no data-editing involved in this step, the local data is guaranteed to be the same as its online version. Eventually the program extracted 855 alloy steels that meet the criteria.

Simple data-processing is performed right after all the data has been saved locally. This step modifies the file names and contents containing non-utf-8 encoding and fixes unwanted line breaks. The data collected from the last step is still string-based, e.g. '97%' is interpreted as a sequence of characters instead of a floating point number. Therefore, it is necessary to convert these strings to machine-readable format (i.e. floating point numbers) before any further data analysis.

These physical variables include: 

|   |  	|  	|  	|  	|  	|
|:-------------------------------:|:---------------------:|:----------------------:|:------------------------:|:---------------:|:---------------------------:|
|density|hardness (Vickers)|thermal conductivity|specific heat capacity|CTE-linear|poisson's ratio|
|electrical resistivity|elongation at break|bulk modulus|modulus of elasticity|shear modulus|tensile strength at yield|

    
    
Ten element types including are considered, including:

| |  |  |  |  |  |  |  |  |  |
|:--------:|--------:|--------:|--------:|--------:|--------:|--------:|--------:|--------:|--------:|
| Fe | Mn | Cr | Ni | Mo | Cu | C | S | Si | P |

Some of the most significant code implemented in this section is shown below.

```python
from selenium import webdriver
from selenium.webdriver.support.ui import Select

# navigate to target website
options = webdriver.ChromeOptions()
driver_path = os.getcwd() + "/chromedriver-linux"
driver = webdriver.Chrome(executable_path=driver_path, chrome_options=options)
driver.get("http://www.matweb.com/index.aspx")
...
# navigate into each alloy page
driver.find_element_by_link_text(name).click()
table = driver.find_element_by_xpath("//table[@class='tabledataformat']")
attrib = []
for row in table.find_elements_by_xpath("//tr[@class='altrow datarowSeparator']"):
    attrib.append([d.text for d in row.find_elements_by_css_selector('td')])
    for row in table.find_elements_by_xpath("//tr[@class=' datarowSeparator']"):
        attrib.append([d.text for d in row.find_elements_by_css_selector('td')])
attrib = np.array(attrib)
...
# collect features from files
data_all_string = []
for filename in os.listdir(ddir):
    instance = ['0%']*len(elem_list) + ['NA']*len(prop_list) # initialize instance row
    data = pd.read_csv(ddir+filename, header=0, index_col=0)
    for ii in range(nfeature):
        if feature_list[ii] in data.index:
            instance[ii] = data.loc[feature_list[ii], 'Metric']
            # get rid of some unexpected '\n' in dataset
            if '\n' in instance[ii]: instance[ii] = instance[ii].replace('\n', ' ') 
    if instance[0] is not 0: data_all_string.append(instance) # steel only
```

## III. Data Analysis

The size of the dataset after featurization is 855$\times$23 (with 855 instances and 23 features). It is quite impractical for humans to directly learn patterns from such a large amount of data. Let's take a look at what we have.

```python
import pandas as pd
data = pd.read_csv('./data_all_float.csv', sep=';', header=0)
data.head()

   Iron, Fe  Carbon, C  Sulfur, S     ...     Shear Modulus  Poissons Ratio  Tensile Strength
0  97.38575      0.300       0.04     ...              80.0            0.29           850.0          
1  95.72250      0.405       0.02     ...              78.0            0.20           786.0           
2  96.46250      0.200       0.04     ...              80.0            0.11           958.0           
3  96.46250      0.200       0.04     ...              80.0            0.45           1125.0         
4  97.38750      0.300       0.04     ...              80.0            0.10           425.0           
...
[855 rows x 23 columns]
```

Using basic statistical knowledge, we decide to start from learning the correlation patten of these properties. The instances with 'NaN' entries are dropped from the dataset, and eventually 254 materials are used to generate the heatmap, as shown below. The upper left corner could be safely ignored since it does not contain useful composition-property information. The lower left part reveals the relationship between elements and different physical properties. The more 'read shift' the block is, the more positive correlation the element has to the property, and vice versa. The lower right region reflects how these physical properties themselves are correlated. This could potentially be interesting as well, but is out of the scope of this study.

Some of the findings agree well with the way elements contribute to alloy steel properties as reported online, e.g. carbon decreases ductility of steel, and could lead to a small shear modulus. However, the information from correlation analysis is more qualitative than quantitative. If we want more quantitative descriptions of the materials composition-property relationship, more sophisticated methods are needed. In the next section we discuss alloy steel data analysis using machine learning models.

```python
import seaborn as sns; sns.set()

data = pd.read_csv("data_all_float.csv", header=0, index_col=None, sep=';')
# compute correlation
corr = data.corr()

# plot correlation heatmap
fig = plt.figure(num=None, figsize=(40, 40), dpi=80, facecolor='w', edgecolor='w')
colormap = sns.diverging_palette(220, 10, as_cmap=True)
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
ylabel = list(corr.columns)
ax = sns.heatmap(corr, cmap=colormap, mask=mask, annot=True, fmt=".2f", 
                 xticklabels=corr.columns[:-1], yticklabels=ylabel)
plt.title(label="Correlation Heatmap of All Features", loc='center', 
          fontdict={'fontname':'DejaVu Sans', 'size':'24', 'color':'black', 
                    'weight':'bold', 'verticalalignment':'bottom'})
plt.show()
```

<img src="CorrelationHeatmap.png" alt="drawing" width="700"/>

## IV. Benchmarking Machine Learning Model Performance

We perform systematic benchmark of different machine learning algorithm performance on various physical property predictions.

Each training takes one physical property (e.g. hardness) as the target, while the other 12 properties plus aforementioned 10 element types are used as input features. Instances with 'NaN' are dropped from the dataset. Both X and y are standardized (i.e. centered to zero and scaled to unit variance) as implemented in StandardScaler() from sklearn.

10 machine learning algorithms are benchmarked here, including:
- Linear Regression	
- Least-angle regression with Lasso (LassoLars)
- Kernel Ridge
- Linear Support Vector Regression (Linear SVR)
- Stochastic Gradient Descent Regression (SGD)
- Multi-layer Perceptron Regressor (MLP)
- AdaBoost Regression
- Random Forest Regression
- Gradient Boosting Regression
- Extreme Gradient Boosting (XGBoost)

The dataset is divided into training (75%) and testing (25%) parts. Grid search method is used for hyper-parameter tuning. The set of hyper-parameters are shown in Supporting Information. The optimal setting of the parameters is determined based on 5-fold cross-validation performed on training data only. R2 score is used as error metric. 

The key part of our implementation is attached below (using XGBoost as an example):

```python
import sys
import xgboost
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import *

# set hyper-parameters
hyper_params = [{
    'n_estimators' : (10, 50, 100, 250, 500, 1000,),
    'learning_rate' : (0.0001,0.01, 0.05, 0.1, 0.2,),
    'gamma' : (0,0.1,0.2,0.3,0.4,),
    'max_depth' : (6,),
    'subsample' : (0.5, 0.75, 1,),
}]

stdscale = StandardScaler()

properties = [ 'Thermal Conductivity', 'Specific Heat Capacity', 'Hardness, Vickers', 
              'Electrical Resistivity', 'Elongation at Break', 'Bulk Modulus', 
              'Modulus of Elasticity', 'Shear Modulus', 'Poissons Ratio', 
              'Tensile Strength, Yield', 'Tensile Strength, Ultimate']

for target in properties:
    X = data.drop(target, axis=1).values
    y = data[target].values
    
    # standardize features
    X = stdscale.fit_transform(X)
    y = stdscale.fit_transform(y.reshape(-1, 1)) 
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, 
                                                        test_size=0.25, random_state=0)
    
    regressor = xgboost.XGBRegressor()

    grid_clf = GridSearchCV(regressor, cv=5, param_grid=hyper_params,
                            verbose=1, n_jobs=8, scoring='r2')
    grid_clf.fit(X_train, y_train.ravel())

    train_score_mse = mean_squared_error(stdscale.inverse_transform(y_train),
                                         stdscale.inverse_transform(grid_clf.predict(X_train)))
    test_score_mse = mean_squared_error(stdscale.inverse_transform(y_test),
                                        stdscale.inverse_transform(grid_clf.predict(X_test)))

    sorted_grid_params = sorted(grid_clf.best_params_.items(), key=operator.itemgetter(0))

    # print results
    out_txt = '\t'.join(['algorithm: ', str(sorted_grid_params).replace('\n', ','), 
                         str(train_score_mse), str(test_score_mse)])

    print(out_txt)
    sys.stdout.flush()
```

The results are plotted below. Training errors are shown in circles and solid lines, test errors are in squares and dashed lines. The y-axis is the calculated root mean square error (RMSE) value divided by the difference of maximum and minimum value in the target data. This normalization step is essential so as to directly compare the performance of different machine learning algorithms in predicting quantities at various scales. The sklearn built-in performance scores (e.g. R2-score) are not used here.

<img src="Benchmark.png" alt="drawing" width="600"/>

Interestingly, some trends can be identified across different algorithms as well as physical properties. In general, linear regression, Lasso, linear SVR and SGD algorithms lead to larger variance in normalized RMSE, while the rest have relatively better performance. XGBoost algorithm achieves the best performance among the 10 benchmarked methods.

The physical properties are divided into two panels due to different scales of RMSE. We find it hard to have a good prediction in hardness and tensile strength, but it may not be a coincidence. It is exciting to notice that from the correlation heatmap analysis, it is clear that hardness is strongly and positively correlated to tensile strengths. This may suggest that these quantities are related to some other factors not considered here, such as processing and microscopic structures. And we need to include the other important features in order to build better machine learning models.

## V. Learning Thermal Properties with XGBoost

We further investigate machine learning model performance on more challenging problems. In the previous section a single property is targeted using both alloy chemical composition and other known physical properties. However, in practical situations, sometimes other properties are also unknown (e.g. a new family of alloy steel). Therefore, our aim here is to further understand the composition-property relationship in alloy steel, i.e. we will only use elemental composition information as input features to predict physical properties. We choose to use Extreme Gradient Boosting (XGBoost) algorithm for further analysis on alloy steel composition-relationship since it outperforms the other algorithms from our benchmark results.

The implementation is essentailly the same as shown previously in the benchmark section and is therefore not presented here. The performance of XGBoost in predicting 11 different physical properties independently below. R2-score is used as error metric in this case. 

<img src="XGBoostR2.png" alt="drawing" width="500"/>

Among the 11 tested properties, 8 of them do not show a strong relationship to composition, both training score and test score are below 0.5. Specific heat capacity and electrical resistivity get a high score in training but score poorly in test. This probably means XGBoost cannot correctly capture the relationship from the training set, but only numerically fit the data. In other words, the model suffers from overfitting. One thing to notice here is that R2-score can be negative, which means the model is even worse than a horizontal line through the mean feature value.

XGBoost performs well in predicting thermal conductivity using composition information, where the test score is even higher than training score. In order to validate our findings, we check the learning curve of our model.

```python
import xgboost
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import StandardScaler

stdscale = StandardScaler()
X = stdscale.fit_transform(X)
y = stdscale.fit_transform(y.reshape(-1, 1)) 

train_sizes, train_scores, valid_scores = learning_curve(xgboost.XGBRegressor(), X, y, 
                                                         train_sizes=list(range(20, 490, 10)), cv=5)
out = []
for ii in range(len(train_sizes)):
    out.append([train_sizes[ii], np.mean(train_scores[ii]), np.mean(valid_scores[ii])])
```

The learning curve of predicting thermal conductivity is shown below. As the number of training samples increase, cross-validation scores quickly increase and converge to the high training score. This clearly means adding more training samples can further improve the performance of XGBoost model in predicting thermal conductivity.

<img src="XGBoostLearncurve.png" alt="drawing" width="500"/>

## VI. Conclusions

We find that machine learning could provide both qualitative and quantitative insights of alloy steel composition-property relationship. Gradient boost-based algorithms outperform the other machine learning algorithms in our benchmark. XGBoost is particularly successful in predicting thermal conductivity using alloy chemical composition, and the performance can be improved by adding more training samples.

## Supporting Information

<center>Table: Hyper-parameters used in different machine learning algorithms for grid search.<center/>

<img src="hyper_param.png" alt="drawing" width="800"/>

P.S. Open to collaboration on data-driven research in materials science and engineering.<br/>
Please contact <raymondwang@u.northwestern.edu> if interested, or visit us at our [group website](https://mtd.mccormick.northwestern.edu/research/)!